# Immoscout24.de Scraper

Ein Script zum dumpen (in `.csv` schreiben) von Immobilien, welche auf [immoscout24.de](http://immoscout24.de) angeboten werden

In [16]:
from bs4 import BeautifulSoup
import json
import urllib2
import random
from random import choice
import time
import geopandas as gpd

In [2]:
# urlquery from Achim Tack. Thank you!
# https://github.com/ATack/GoogleTrafficParser/blob/master/google_traffic_parser.py
def urlquery(url):
    # function cycles randomly through different user agents and time intervals to simulate more natural queries
    try:
        sleeptime = float(random.randint(1,6))/5
        time.sleep(sleeptime)

        agents = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17',
        'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
        'Opera/12.80 (Windows NT 5.1; U; en) Presto/2.10.289 Version/12.02',
        'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
        'Mozilla/3.0',
        'Mozilla/5.0 (iPhone; U; CPU like Mac OS X; en) AppleWebKit/420+ (KHTML, like Gecko) Version/3.0 Mobile/1A543a Safari/419.3',
        'Mozilla/5.0 (Linux; U; Android 0.5; en-us) AppleWebKit/522+ (KHTML, like Gecko) Safari/419.3',
        'Opera/9.00 (Windows NT 5.1; U; en)']

        agent = choice(agents)
        opener = urllib2.build_opener()
        opener.addheaders = [('User-agent', agent)]
        #print agent

        html = opener.open(url).read()
        time.sleep(sleeptime)
        
        return html

    except:
        print "fehler in urlquery"

In [3]:
def immoscout24parser(url):
    
    ''' Parser holt aus Immoscout24.de Suchergebnisseiten die Immobilien '''
    
    try:
        soup = BeautifulSoup(urlquery(url), 'html.parser')
        scripts = soup.findAll('script')
        for script in scripts:
            #print script.text.strip()
            if 'IS24.resultList' in script.text.strip():
                s = script.string.split('\n')
                for line in s:
                    #print('\n\n\'%s\'' % line)
                    if line.strip().startswith('resultListModel'):
                        resultListModel = line.strip('resultListModel: ')
                        immo_json = json.loads(resultListModel[:-1])

                        searchResponseModel = immo_json[u'searchResponseModel']
                        resultlist_json = searchResponseModel[u'resultlist.resultlist']
                        
                        return resultlist_json

    except Exception, e:
        print "fehler in immoscout24 parser: %s" % e

## Main Loop

Geht Wohnungen und Häuser, jeweils zum Kauf und Miete durch und sammelt die Daten

In [4]:
immos = {}

b = 'Berlin'
s = 'Berlin'
k = 'Wohnung'
w = 'Miete'

page = 0
print('Suche %s / %s' % (k, w))

while True:
    page+=1
    url = 'http://www.immobilienscout24.de/Suche/S-T/P-%s/%s-%s/%s/%s?pagerReporting=true' % (page, k, w, b, s)

    # Because of some timeout or immoscout24.de errors,
    # we try until it works \o/
    resultlist_json = None
    while resultlist_json is None:
        try:
            resultlist_json = immoscout24parser(url)
            numberOfPages = int(resultlist_json[u'paging'][u'numberOfPages'])
            pageNumber = int(resultlist_json[u'paging'][u'pageNumber'])
        except:
            pass

    if page>numberOfPages:
        break

    # Get the data
    for resultlistEntry in resultlist_json['resultlistEntries'][0][u'resultlistEntry']:
        realEstate_json = resultlistEntry[u'resultlist.realEstate']
        
        realEstate = {}

        realEstate[u'Miete/Kauf'] = w
        realEstate[u'Haus/Wohnung'] = k

        realEstate['address'] = realEstate_json['address']['description']['text']
        realEstate['city'] = realEstate_json['address']['city']
        realEstate['postcode'] = realEstate_json['address']['postcode']
        realEstate['quarter'] = realEstate_json['address']['quarter']
        try:
            realEstate['lat'] = realEstate_json['address'][u'wgs84Coordinate']['latitude']
            realEstate['lon'] = realEstate_json['address'][u'wgs84Coordinate']['longitude']
        except:
            realEstate['lat'] = None
            realEstate['lon'] = None
            
        realEstate['title'] = realEstate_json['title']

        realEstate['numberOfRooms'] = realEstate_json['numberOfRooms']
        realEstate['livingSpace'] = realEstate_json['livingSpace']
        
        realEstate['balcony'] = realEstate_json['balcony']
        realEstate['builtInKitchen'] = realEstate_json['builtInKitchen']
        realEstate['garden'] = realEstate_json['garden']
        realEstate['price'] = realEstate_json['price']['value']
        realEstate['privateOffer'] = realEstate_json['privateOffer']
        
        realEstate['floorplan'] = realEstate_json['floorplan']
        realEstate['from'] = realEstate_json['companyWideCustomerId']
        realEstate['ID'] = realEstate_json[u'@id']
        realEstate['url'] = u'https://www.immobilienscout24.de/expose/%s' % realEstate['ID']

        immos[realEstate['ID']] = realEstate

    print('Scrape Page %i/%i (%i Immobilien %s %s gefunden)' % (page, numberOfPages, len(immos), k, w))

Suche Wohnung / Miete
Scrape Page 1/167 (20 Immobilien Wohnung Miete gefunden)
Scrape Page 2/167 (40 Immobilien Wohnung Miete gefunden)
Scrape Page 3/167 (60 Immobilien Wohnung Miete gefunden)
Scrape Page 4/167 (80 Immobilien Wohnung Miete gefunden)
Scrape Page 5/167 (100 Immobilien Wohnung Miete gefunden)
Scrape Page 6/167 (120 Immobilien Wohnung Miete gefunden)
Scrape Page 7/167 (140 Immobilien Wohnung Miete gefunden)
Scrape Page 8/167 (160 Immobilien Wohnung Miete gefunden)
Scrape Page 9/167 (180 Immobilien Wohnung Miete gefunden)
Scrape Page 10/167 (200 Immobilien Wohnung Miete gefunden)
Scrape Page 11/167 (220 Immobilien Wohnung Miete gefunden)
Scrape Page 12/167 (240 Immobilien Wohnung Miete gefunden)
Scrape Page 13/167 (260 Immobilien Wohnung Miete gefunden)
Scrape Page 14/167 (280 Immobilien Wohnung Miete gefunden)
Scrape Page 15/167 (300 Immobilien Wohnung Miete gefunden)
Scrape Page 16/167 (320 Immobilien Wohnung Miete gefunden)
Scrape Page 17/167 (340 Immobilien Wohnung Miet

## Datenaufbereitung & Cleaning

Die gesammelten Daten werden in ein sauberes Datenformat konvertiert, welches z.B. auch mit Excel gelesen werden kann. Weiterhin werden die Ergebnisse pseudonymisiert, d.h. die Anbieter bekommen eindeutige Nummern statt Klarnamen.

In [5]:
import datetime
timestamp = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d-%H-%M')

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(immos).T
df.index.name = 'ID'

In [8]:
len(df)

3319

In [9]:
df.head()

,Haus/Wohnung,ID,Miete/Kauf,address,balcony,builtInKitchen,city,floorplan,from,garden,lat,livingSpace,lon,numberOfRooms,postcode,price,privateOffer,quarter,title,url
ID,,,,,,,,,,,,,,,,,,,,
100027976,Wohnung,100027976,Miete,"Pohlestraße 14, Köpenick (Köpenick), Berlin",true,true,Berlin,true,001.3112839,false,52.4406,83,13.5841,3,12557,828,false,Köpenick (Köpenick),3 RaumWohnung; Balkon +Dach Garten Anteil + EB...,https://www.immobilienscout24.de/expose/100027976
100122011,Wohnung,100122011,Miete,"Soldiner Straße 35, Wedding (Wedding), Berlin",false,false,Berlin,false,001.172849,false,52.5599,83.55,13.3828,2,13359,1086,false,Wedding (Wedding),!!! DREI NETTOKALTMIETEN FREI !!! - Große 2-Zi...,https://www.immobilienscout24.de/expose/100122011
100149210,Wohnung,100149210,Miete,"Vopeliuspfad 10, Zehlendorf (Zehlendorf), Berlin",true,true,Berlin,true,001.453607,false,52.4349,238,13.2673,6,14169,2760,false,Zehlendorf (Zehlendorf),++Eindrucksvolle Altbauwohnung im 2. OG mit 3 ...,https://www.immobilienscout24.de/expose/100149210
100150862,Wohnung,100150862,Miete,"Rudolf-von-Gneist-Gasse 1, Tiergarten (Tiergar...",true,true,Berlin,true,001.37030,false,52.5068,102.2,13.3742,2,10785,1636,false,Tiergarten (Tiergarten),#Skylights: Dachgeschossmaisonette mit großer ...,https://www.immobilienscout24.de/expose/100150862
100155473,Wohnung,100155473,Miete,"Wackenbergstr. 57, Niederschönhausen (Pankow),...",true,true,Berlin,true,009.bfab7c1d-44dd-4f47-bd7f-06d43ad6fd52,false,52.5862,74.94,13.4138,2,13156,636.76,true,Niederschönhausen (Pankow),"Schön Wohnen im Erdgeschoss mit Terrasse und ""...",https://www.immobilienscout24.de/expose/100155473


In [10]:
df['price_sq_m'] = df['price'] / df['livingSpace']

In [11]:
df.head()

,Haus/Wohnung,ID,Miete/Kauf,address,balcony,builtInKitchen,city,floorplan,from,garden,...,livingSpace,lon,numberOfRooms,postcode,price,privateOffer,quarter,title,url,price_sq_m
ID,,,,,,,,,,,,,,,,,,,,,
100027976,Wohnung,100027976,Miete,"Pohlestraße 14, Köpenick (Köpenick), Berlin",true,true,Berlin,true,001.3112839,false,...,83,13.5841,3,12557,828,false,Köpenick (Köpenick),3 RaumWohnung; Balkon +Dach Garten Anteil + EB...,https://www.immobilienscout24.de/expose/100027976,9.9759
100122011,Wohnung,100122011,Miete,"Soldiner Straße 35, Wedding (Wedding), Berlin",false,false,Berlin,false,001.172849,false,...,83.55,13.3828,2,13359,1086,false,Wedding (Wedding),!!! DREI NETTOKALTMIETEN FREI !!! - Große 2-Zi...,https://www.immobilienscout24.de/expose/100122011,12.9982
100149210,Wohnung,100149210,Miete,"Vopeliuspfad 10, Zehlendorf (Zehlendorf), Berlin",true,true,Berlin,true,001.453607,false,...,238,13.2673,6,14169,2760,false,Zehlendorf (Zehlendorf),++Eindrucksvolle Altbauwohnung im 2. OG mit 3 ...,https://www.immobilienscout24.de/expose/100149210,11.5966
100150862,Wohnung,100150862,Miete,"Rudolf-von-Gneist-Gasse 1, Tiergarten (Tiergar...",true,true,Berlin,true,001.37030,false,...,102.2,13.3742,2,10785,1636,false,Tiergarten (Tiergarten),#Skylights: Dachgeschossmaisonette mit großer ...,https://www.immobilienscout24.de/expose/100150862,16.0078
100155473,Wohnung,100155473,Miete,"Wackenbergstr. 57, Niederschönhausen (Pankow),...",true,true,Berlin,true,009.bfab7c1d-44dd-4f47-bd7f-06d43ad6fd52,false,...,74.94,13.4138,2,13156,636.76,true,Niederschönhausen (Pankow),"Schön Wohnen im Erdgeschoss mit Terrasse und ""...",https://www.immobilienscout24.de/expose/100155473,8.49693


In [12]:
df.columns

Index([  u'Haus/Wohnung',             u'ID',     u'Miete/Kauf',
              u'address',        u'balcony', u'builtInKitchen',
                 u'city',      u'floorplan',           u'from',
               u'garden',            u'lat',    u'livingSpace',
                  u'lon',  u'numberOfRooms',       u'postcode',
                u'price',   u'privateOffer',        u'quarter',
                u'title',            u'url',     u'price_sq_m'],
      dtype='object')

In [14]:
df.shape

(3319, 21)

## Alles Dumpen

In [13]:
f = open('%s-%s-%s.csv' % (timestamp, k, w), 'wb')
f.write('# %s %s from immoscout24.de on %s\n' % (k,w,timestamp))
df[(df['Haus/Wohnung']==k) & (df['Miete/Kauf']==w)].to_csv(f, encoding='utf-8')
f.close()

In [11]:
#df.to_excel('%s-%s-%s.xlsx' % (timestamp, k, w))

Fragen? [@Balzer82](https://twitter.com/Balzer82)